In [0]:
import numpy as np
import pandas as pd
import re
import math
import time
from google.colab import drive

In [2]:
tensorflow_version

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [0]:
import tensorflow as tf

try:
    
    %tensorflow_version 2.x
except Exception:
    pass

from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [4]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
with open("/content/drive/My Drive/Transformer/english.en",mode = 'r',encoding='utf-8')as f:
    english = f.read()

with open("/content/drive/My Drive/Transformer/spanish.es",mode='r',encoding='utf-8')as f:
    spanish = f.read() 

with open("/content/drive/My Drive/Transformer/nonbreaking_prefix.en",mode='r',encoding='utf-8')as f:
    nonbreaking_english = f.read() 

with open("/content/drive/My Drive/Transformer/nonbreaking_prefix.es",mode='r',encoding='utf-8')as f:
    nonbreaking_spanish = f.read()

In [6]:
english[:50]

'Resumption of the session\nI declare resumed the se'

In [7]:
spanish[:50]

'Reanudación del período de sesiones\nDeclaro reanud'

In [8]:
nonbreaking_english

'\nA\nB\nC\nD\nE\nF\nG\nH\nI\nJ\nK\nL\nM\nN\nO\nP\nQ\nR\nS\nT\nU\nV\nW\nX\nY\nZ\nAdj\nAdm\nAdv\nAsst\nBart\nBldg\nBrig\nBros\nCapt\nCmdr\nCol\nComdr\nCon\nCorp\nCpl\nDR\nDr\nDrs\nEns\nGen\nGov\nHon\nHr\nHosp\nInsp\nLt\nMM\nMR\nMRS\nMS\nMaj\nMessrs\nMlle\nMme\nMr\nMrs\nMs\nMsgr\nOp\nOrd\nPfc\nPh\nProf\nPvt\nRep\nReps\nRes\nRev\nRt\nSen\nSens\nSfc\nSgt\nSr\nSt\nSupt\nSurg\nv\nvs\ni.e\nrev\ne.g\nRs\nNo \nNos\nArt\nNr\npp \nJan\nFeb\nMar\nApr\nJun\nJul\nAug\nSep\nOct\nNov\nDec\n'

In [0]:
nonbreaking_english = nonbreaking_english.split("\n")
nonbreaking_english = [" " + prefix + "." for prefix in nonbreaking_english]

nonbreaking_spanish = nonbreaking_spanish.split("\n")
nonbreaking_spanish = [" " + prefix + "." for prefix in nonbreaking_spanish]

In [10]:
nonbreaking_english

[' .',
 ' A.',
 ' B.',
 ' C.',
 ' D.',
 ' E.',
 ' F.',
 ' G.',
 ' H.',
 ' I.',
 ' J.',
 ' K.',
 ' L.',
 ' M.',
 ' N.',
 ' O.',
 ' P.',
 ' Q.',
 ' R.',
 ' S.',
 ' T.',
 ' U.',
 ' V.',
 ' W.',
 ' X.',
 ' Y.',
 ' Z.',
 ' Adj.',
 ' Adm.',
 ' Adv.',
 ' Asst.',
 ' Bart.',
 ' Bldg.',
 ' Brig.',
 ' Bros.',
 ' Capt.',
 ' Cmdr.',
 ' Col.',
 ' Comdr.',
 ' Con.',
 ' Corp.',
 ' Cpl.',
 ' DR.',
 ' Dr.',
 ' Drs.',
 ' Ens.',
 ' Gen.',
 ' Gov.',
 ' Hon.',
 ' Hr.',
 ' Hosp.',
 ' Insp.',
 ' Lt.',
 ' MM.',
 ' MR.',
 ' MRS.',
 ' MS.',
 ' Maj.',
 ' Messrs.',
 ' Mlle.',
 ' Mme.',
 ' Mr.',
 ' Mrs.',
 ' Ms.',
 ' Msgr.',
 ' Op.',
 ' Ord.',
 ' Pfc.',
 ' Ph.',
 ' Prof.',
 ' Pvt.',
 ' Rep.',
 ' Reps.',
 ' Res.',
 ' Rev.',
 ' Rt.',
 ' Sen.',
 ' Sens.',
 ' Sfc.',
 ' Sgt.',
 ' Sr.',
 ' St.',
 ' Supt.',
 ' Surg.',
 ' v.',
 ' vs.',
 ' i.e.',
 ' rev.',
 ' e.g.',
 ' Rs.',
 ' No .',
 ' Nos.',
 ' Art.',
 ' Nr.',
 ' pp .',
 ' Jan.',
 ' Feb.',
 ' Mar.',
 ' Apr.',
 ' Jun.',
 ' Jul.',
 ' Aug.',
 ' Sep.',
 ' Oct.',
 ' Nov.',
 '

In [0]:
corpus_en = english
## most of cleaning part is done by Tokenizer we will be using
# What we must do is remove '.' from non ending words such as i.e. or a.m.
for prefix in nonbreaking_english:
    
    corpus_en = corpus_en.replace(prefix, prefix+"$$$")
  # here now nonending "." will be denoted by $$$ at last

corpus_en = re.sub(r"\.(?=[0-9]|[A-Z]|[a-z])","$$$",corpus_en)
#"." have separate meaning in regex so we have to use "\" before it
#"?=" means check for the regular expression but dont replace "|"== or
# now we also want to add $$$ if "." is not followed by space as it doesnt indicate end

corpus_en = re.sub(r"\.$$$"," ",corpus_en)
corpus_en = re.sub(r"  +"," ",corpus_en)
corpus_en = corpus_en.split("\n")

corpus_es = spanish
for prefix in nonbreaking_spanish:
    corpus_es = corpus_es.replace(prefix, prefix+"$$$")
corpus_es = re.sub(r"\.(?=[0-9]|[A-Z]|[a-z])","$$$",corpus_es)
corpus_es = re.sub(r"\.$$$"," ",corpus_es)
corpus_es = re.sub(r"  +"," ",corpus_es)
corpus_es = corpus_es.split("\n")

In [12]:
corpus_en[:200]

['Resumption of the session',
 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.',
 "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.",
 'You have requested a debate on this subject in the course of the next few days, during this part-session.',
 "In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.",
 "Please rise, then, for this minute' s silence.",
 "(The House rose and observed a minute' s silence)",
 'Madam President, on a point of order.',
 'You will be aware from the press and television that there have be

In [0]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_en,target_vocab_size=2**14)
tokenizer_es = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_es,target_vocab_size=2**14)

In [0]:
VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2
VOCAB_SIZE_ES = tokenizer_es.vocab_size + 2
# 2 is added as we add 2 new token i.e. for beggining and ending of sentence.


In [0]:
inputs = [[VOCAB_SIZE_EN-2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN-1]
          for sentence in corpus_en]
outputs = [[VOCAB_SIZE_ES-2] + tokenizer_es.encode(sentence) + [VOCAB_SIZE_ES-1]
           for sentence in corpus_es]
#vocab_size_en-2 make it as orginal vocab size so we dont use that in encoding phase
#ends at vocab_size - 1

In [0]:
 #now we remove long sentences cause it requires huge computation power and time
MAX_LEN = 20 
idx_to_remove = [count for count,sen in enumerate(inputs) if len(sen)>MAX_LEN]

for idx in reversed(idx_to_remove):
    
  #reverse order ma garnu parxa natra suru ko remove garda mathi ko sabai ko index ghatxa
    del inputs[idx]
    del outputs[idx]

idx_to_remove1 = [count for count,sen in enumerate(outputs) if len(sen)>MAX_LEN]

for idx in reversed(idx_to_remove1):
    del inputs[idx]
    del outputs[idx]



In [0]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                       value=0,
                                                       padding='post',
                                                       maxlen = MAX_LEN)
outputs = tf.keras.preprocessing.sequence.pad_sequences(outputs,
                                                       value=0,
                                                       padding='post',
                                                       maxlen = MAX_LEN)

In [0]:
BATCH_SIZE = 64
BUFFER_SIZE =20000
#create datasets
dataset = tf.data.Dataset.from_tensor_slices((inputs,outputs))
dataset = dataset.cache() 
#does nothing to improve performance but increase speed of training by making data available easily
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
#also just improves speed

Positional encoding formulae:

PE(pos,2i)=sin(pos/10000**(2i/dmodel)) 

PE(pos,2i+1)=cos(pos/10000**(2i/dmodel))

In [0]:
class PositionalEncoding(layers.Layer):
    def __init__(self):
        super(PositionalEncoding,self).__init__()
    #yesma just computation matra hunxa kei update garna pardaina so just super matra hunxa.
    def get_angles(self,pos,i,d_model): #pos = [seq_model,1] and i=[1,d_model]
    #pos and i are array
        angles = 1/np.power(10000.,(2*(i//2))/np.float32(d_model))
    #here in above formula we can see sine is for even and cos is for odd
    #and i doesnt represent i but 2i in sin so we // by 2
        return pos * angles #matrix of size (seq_length,d_model)
  
    def call(self,inputs):
        seq_len = inputs.shape.as_list()[-2]
        d_model = inputs.shape.as_list()[-1]
        angles = self.get_angles(np.arange(seq_len)[:,np.newaxis],
                             np.arange(d_model)[np.newaxis,:],
                             d_model) #np.newaxis le dimension badhauxa say if (28,) xa vane (28,1)
        angles[:, 0::2] = np.sin(angles[:, 0::2])
        angles[:, 1::2] = np.cos(angles[:, 1::2])
        pos_encoding = angles[np.newaxis,...] # ... means takes all
        return inputs + tf.cast(pos_encoding,tf.float32)


In [0]:
def scale_dot_product_attention(queries,keys,values,mask):
    product = tf.matmul(queries,keys,transpose_b=True)

    key_dim = tf.cast(tf.shape(keys)[-1], tf.float32)
  # -1 gives last term i.e. embedding
    scaled_product = product/ tf.math.sqrt(key_dim)

    if mask is not None:
    scaled_product += (mask* -1e9)
  #it was optional but still we put mask
  # it is placed negative infinity such that when placed on softmax gives 0 i.e. no effect.

    attention = tf.matmul(tf.nn.softmax(scaled_product,axis=-1),values) #softmax is applied to last axis

    return attention

In [0]:
class MultiHeadAttention(layers.Layer):
    def __init__(self,nb_proj):
        super(MultiHeadAttention,self).__init__()
        self.nb_proj = nb_proj
  #we require to define all _lin function and we know they all have same no. of units i.e. dmodel
  #but we dont have acces to d_model
  #so we can get that acess by defining build method
  # which is similar to __init__ but instead of being called when object is initialized it is called when first time object is used.

    def build(self,input_shape):
        self.d_model = input_shape[-1]
        assert self.d_model % self.nb_proj == 0
    # assert demands to have that condition
        self.d_proj = self.d_model// self.nb_proj
        self.queries_lin = layers.Dense(units=self.d_model)
        self.keys_lin = layers.Dense(units=self.d_model)
        self.values_lin = layers.Dense(units=self.d_model)
        self.final_lin = layers.Dense(units=self.d_model)

    def split_proj(self,inputs,batch_size): #inputs = [batch_size,seq_length,d_model]size
        shaped = [batch_size,-1,self.nb_proj,self.d_proj] # -1 gives seq-len
        splited_inputs = tf.reshape(inputs,shape=shaped) # [bat_size,seq_len,nb_proj,d_proj]
        return tf.transpose(splited_inputs,perm = [0, 2, 1, 3]) #[bat_size,nb_proj,seq_len,d_proj]
 #d_proj = d_model//nb_proj
    def call(self,queries,keys,values,mask):
        batch_size = tf.shape(queries)[0]
        queries = self.queries_lin(queries)
        keys = self.keys_lin(keys)
        values = self.values_lin(values)

        queries = self.split_proj(queries,batch_size)
        keys = self.split_proj(keys,batch_size)
        values = self.split_proj(values,batch_size)

        attention = scale_dot_product_attention(queries, keys, values, mask)
    # concat is exactly opposite to split proj   
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention,
                                      shape=(batch_size, -1, self.d_model))
        outputs = self.final_lin(concat_attention)
        return outputs

In [0]:
class EncodingLayer(layers.Layer):
    def __init__(self,FFN_units,nb_proj,dropout_rate):
        super(EncodingLayer,self).__init__()
        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout_rate = dropout_rate

    def build(self,input_shape):
        self.d_model = input_shape[-1]
        self.multi_head_attention = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate= self.dropout_rate)
        self.norm_1 = layers.LayerNormalization(epsilon =1e-6)
        self.dense_1 = layers.Dense(units= self.FFN_units,activation="relu")
        self.dense_2 = layers.Dense(units = self.d_model)
        self.dropout_2 = layers.Dropout(rate= self.dropout_rate)

        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)


    def call(self,inputs,mask,training): #training is boolean here it is used for dropout as it is applied only while training
        attention = self.multi_head_attention(inputs,
                                          inputs,
                                          inputs,
                                          mask)
        attention = self.dropout_1(attention, training= training)
        attention = self.norm_1(attention+inputs)

        outputs = self.dense_1(attention)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_2(outputs, training= training)
        outputs = self.norm_2(outputs+attention)

         return outputs


In [0]:
class Encoder(layers.Layer):
    def __init__(self,nb_layers,FFN_units,nb_proj,dropout_rate,vocab_size,d_model,name="encoder"):
        super(Encoder,self).__init__(name=name)
        self.d_model = d_model
        self.nb_layers = nb_layers

        self.embedding = layers.Embedding(vocab_size,d_model)
        self.positional_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout_rate)
        self.encoding_layers = [EncodingLayer(FFN_units,nb_proj,dropout_rate) for _ in range(nb_layers)]

    def call(self,inputs,mask,training):
        outputs = self.embedding(inputs)
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    #just normalization ko lagi sqrt(d_model) le multiply garne vanethyo paper ma
        outputs = self.positional_encoding(outputs)
        outputs = self.dropout(outputs,training)

        for i in range(self.nb_layers):
            outputs = self.encoding_layers[i](outputs,mask,training)
        return outputs

In [0]:
class DecoderLayer(layers.Layer):
    
    def __init__(self,FFN_units,nb_proj,dropout_rate):
        super(DecoderLayer,self).__init__()
        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout_rate = dropout_rate

    def build(self,input_shape):
        self.d_model = input_shape[-1]

        self.multi_head_attention_1 = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate = self.dropout_rate)
        self.norm_1 = layers.LayerNormalization(epsilon =1e-6)

        self.multi_head_attention_2 = MultiHeadAttention(self.nb_proj)
        self.dropout_2 = layers.Dropout(rate = self.dropout_rate)
        self.norm_2 = layers.LayerNormalization(epsilon =1e-6)
    
        self.dense_1 = layers.Dense(units= self.FFN_units,activation="relu")
        self.dense_2 = layers.Dense(units= self.d_model)
        self.dropout_3 = layers.Dropout(rate = self.dropout_rate)
        self.norm_3 = layers.LayerNormalization(epsilon =1e-6)
  
    def call(self,inputs,enc_outputs,mask_1,mask_2,training):# 2 mask for 2 attention 
        attention1 = self.multi_head_attention_1(inputs,inputs,inputs,mask_1)
        attention1 = self.dropout_1(attention1,training)
        attention1 = self.norm_1(attention1+inputs)

        attention2 = self.multi_head_attention_2(attention1,enc_outputs,enc_outputs,mask_2)
        attention2 = self.dropout_2(attention2,training)
        attention2 = self.norm_2(attention2+attention1)

        outputs = self.dense_1(attention2)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_3(outputs,training)
        outputs = self.norm_3(outputs + attention2)

        return outputs
    


In [0]:
class Decoder(layers.Layer):
    
    def __init__(self,
                 nb_layers,
                 FFN_units,
                 nb_proj,
                 dropout_rate,
                 vocab_size,
                 d_model,
                 name="decoder"):
        super(Decoder, self).__init__(name=name)
        self.d_model = d_model
        self.nb_layers = nb_layers
        
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout_rate)
        
        self.dec_layers = [DecoderLayer(FFN_units,
                                        nb_proj,
                                        dropout_rate) 
                           for i in range(nb_layers)]
    
    def call(self, inputs, enc_outputs, mask_1, mask_2, training):
        outputs = self.embedding(inputs)
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        outputs = self.pos_encoding(outputs)
        outputs = self.dropout(outputs, training)
        
        for i in range(self.nb_layers):
            outputs = self.dec_layers[i](outputs,
                                         enc_outputs,
                                         mask_1,
                                         mask_2,
                                         training)

        return outputs

In [0]:
class Transformer(layers.Layer):
    def __init__(self,vocab_size_enc,vocab_size_dec,d_model,nb_layers,FFN_units,nb_proj,
               dropout_rate,name="transformer"):
        super(Transformer,self).__init__()  
        self.encoder = Encoder(nb_layers,FFN_units,nb_proj,dropout_rate,vocab_size_enc,d_model)
        self.decoder = Decoder(nb_layers,FFN_units,nb_proj,dropout_rate,vocab_size_dec,d_model)
        self.last_linear = layers.Dense(units= vocab_size_dec,name="lin_output")

    def create_padding_mask(self,seq): #seq = [ batch_size,seq_len]
        mask= tf.cast(tf.math.equal(seq,0),tf.float32)
        return mask[:,tf.newaxis,tf.newaxis,:]
    #here we add two dimension
    #one for saying that mask is individually for sub-sequences
    #another is just to make sure we can use row as many time as wanted

    def create_look_ahead_mask(self,seq):
        seq_len = tf.shape(seq)[1]
        look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len,seq_len)),-1,0) #lower triangular matrix
        return look_ahead_mask
  # we use look a head matrix along with padding matrix with tf.maximum and it will automatically increase the dimension as required.

    def call(self,enc_inputs,dec_inputs, training):
        enc_mask = self.create_padding_mask(enc_inputs)

        dec_mask_1 = tf.maximum(self.create_padding_mask(dec_inputs),
                            self.create_look_ahead_mask(dec_inputs))
    
        dec_mask_2 = self.create_padding_mask(enc_inputs)

        enc_outputs = self.encoder(enc_inputs,enc_mask, training)

        dec_outputs = self.decoder(dec_inputs,enc_outputs,dec_mask_1,dec_mask_2,training)

        outputs = self.last_linear(dec_outputs)
        return outputs

In [0]:
tf.keras.backend.clear_session()

# Hyper-parameters
D_MODEL = 128 # 512
NB_LAYERS = 4 # 6
FFN_UNITS = 512 # 2048
NB_PROJ = 8 # 8
DROPOUT_RATE = 0.2# 0.1

transformer = Transformer(vocab_size_enc=VOCAB_SIZE_EN,
                          vocab_size_dec=VOCAB_SIZE_ES,
                          d_model=D_MODEL,
                          nb_layers=NB_LAYERS,
                          FFN_units=FFN_UNITS,
                          dropout_rate=DROPOUT_RATE,
                          nb_proj=NB_PROJ
                          )

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction="none")
#reduction none means we dont want to take loss of all batch to represent by one numeric value
def loss_function(target, pred):
    mask = tf.math.logical_not(tf.math.equal(target, 0))
    #first we want to get rid of losses from padding tokens
    #above code we will get rid of all the loss =0 in targets i.e. padding tokens 
    loss_ = loss_object(target, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)
    #in this way we compute loss of only rreal number not the padding tokens

train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

In [0]:
# as mentioned in paper learning rate is self customized
#so we customize as follow
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self,d_model,warmup_steps=4000):
        super(CustomSchedule,self).__init__()

        self.d_model = tf.cast(d_model,tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self,step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step* (self.warmup_steps**-1.5)
    # since by default we put warmup_step = 4000 so for first 4000 arg2 is lower and follow them and then arg1  is lower
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1,arg2)

    learning_rate = CustomSchedule(D_MODEL)

    optimizer = tf.keras.optimizers.Adam(learning_rate,
                                     beta_1=0.9,
                                     beta_2=0.98,
                                     epsilon=1e-9)
# values are from paper

In [0]:
checkpoint_path = "./drive/My Drive/Transformer/ckpt/"
ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [0]:
EPOCHS = 10
for epoch in range(EPOCHS):
    print("Start of epoch {}".format(epoch+1))
    start = time.time()
    #reset loss and accuracy before each epochs
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for (batch, (enc_inputs, targets)) in enumerate(dataset):
      # send in tuples as dataset have inputs and targets
        dec_inputs = targets[:, :-1] 
        # all targets except last
        dec_outputs_real = targets[:, 1:]
        # excluding 1st one as it is starting tokens
        with tf.GradientTape() as tape:
          #GradientTape is used to track all the info while training
            predictions = transformer(enc_inputs, dec_inputs, training=True)
            loss = loss_function(dec_outputs_real, predictions)
        
        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
        
        train_loss(loss)
        train_accuracy(dec_outputs_real, predictions)
        
        if batch % 50 == 0:
            print("Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}".format(
                epoch+1, batch, train_loss.result(), train_accuracy.result()))
            
    ckpt_save_path = ckpt_manager.save()
    print("Saving checkpoint for epoch {} at {}".format(epoch+1,
                                                        ckpt_save_path))
    print("Time taken for 1 epoch: {} secs\n".format(time.time() - start))

Start of epoch 1
Epoch 1 Batch 0 Loss 1.6437 Accuracy 0.3692
Epoch 1 Batch 50 Loss 1.5911 Accuracy 0.3914
Epoch 1 Batch 100 Loss 1.5977 Accuracy 0.3944
Epoch 1 Batch 150 Loss 1.5863 Accuracy 0.3942
Epoch 1 Batch 200 Loss 1.5903 Accuracy 0.3955
Epoch 1 Batch 250 Loss 1.5888 Accuracy 0.3960
Epoch 1 Batch 300 Loss 1.5848 Accuracy 0.3957
Epoch 1 Batch 350 Loss 1.5835 Accuracy 0.3964
Epoch 1 Batch 400 Loss 1.5795 Accuracy 0.3957
Epoch 1 Batch 450 Loss 1.5782 Accuracy 0.3958
Epoch 1 Batch 500 Loss 1.5768 Accuracy 0.3957
Epoch 1 Batch 550 Loss 1.5729 Accuracy 0.3962
Epoch 1 Batch 600 Loss 1.5708 Accuracy 0.3963
Epoch 1 Batch 650 Loss 1.5684 Accuracy 0.3964
Epoch 1 Batch 700 Loss 1.5679 Accuracy 0.3960
Epoch 1 Batch 750 Loss 1.5673 Accuracy 0.3962
Epoch 1 Batch 800 Loss 1.5674 Accuracy 0.3963
Epoch 1 Batch 850 Loss 1.5623 Accuracy 0.3967
Epoch 1 Batch 900 Loss 1.5601 Accuracy 0.3972
Epoch 1 Batch 950 Loss 1.5543 Accuracy 0.3978
Epoch 1 Batch 1000 Loss 1.5500 Accuracy 0.3991
Epoch 1 Batch 1050 

In [0]:
def evaluate(inp_sentence):
    inp_sentence = \
        [VOCAB_SIZE_EN-2] + tokenizer_en.encode(inp_sentence) + [VOCAB_SIZE_EN-1]
    enc_input = tf.expand_dims(inp_sentence, axis=0)
    # to simulate batch size we add dim at axis =0
    output = tf.expand_dims([VOCAB_SIZE_ES-2], axis=0)
    
    for _ in range(MAX_LENGTH):
        predictions = transformer(enc_input, output, False)
        #no dropout apply to testing so False
        prediction = predictions[:, -1:, :]
        #we take from last term so -1:
        predicted_id = tf.cast(tf.argmax(prediction, axis=-1), tf.int32)
        
        if predicted_id == VOCAB_SIZE_ES-1:
          #if predicted id is end token of spanish word
            return tf.squeeze(output, axis=0)
        # we cant directly return output as we have added dimension for batch size
        output = tf.concat([output, predicted_id], axis=-1)
        
    return tf.squeeze(output, axis=0)

In [0]:
def translate(sentence):
    output = evaluate(sentence).numpy()
    # sentence is encoded so  we directly evaluate
    # it will be in tensor so for our convinent we convert it to numpy
    predicted_sentence = tokenizer_fr.decode(
        [i for i in output if i < VOCAB_SIZE_ES-2]
    )
    # output is in encoded term so we decode it but not all , only real sentences token
    
    print("Input: {}".format(sentence))
    print("Predicted translation: {}".format(predicted_sentence))

In [0]:
translate("This is a really powerful tool!")